In [39]:
import pandas as pd
from datetime import datetime, time, timedelta

In [40]:
def unroll_distance_matrix(df)->pd.DataFrame():
    # Initialize an empty list to store DataFrames
    dfs = []

    # Iterate over the rows of the distance matrix
    for i, row in df.iterrows():
        id_start = row.name  # Row index is the id_start
        for id_end, distance in row.items():
            if id_start != id_end:  # Exclude same id_start to id_end
                # Append to the list of DataFrames
                dfs.append(pd.DataFrame({'id_start': [id_start], 'id_end': [id_end], 'distance': [distance]}))

    # Concatenate all DataFrames into a single DataFrame
    unrolled_df = pd.concat(dfs, ignore_index=True)

    return unrolled_df

In [41]:
def calculate_time_based_toll_rates(df):
    df['start_time'] = pd.to_datetime(df['start_time'], format='%H:%M:%S').dt.time
    df['end_time'] = pd.to_datetime(df['end_time'], format='%H:%M:%S').dt.time
    df['start_day'] = df['start_time'].dt.day_name()
    df['end_day'] = df['end_time'].dt.day_name()

    def apply_discount_factor(row):
        start_time = row['start_time']
        end_time = row['end_time']
        start_day = row['start_day']
        end_day = row['end_day']

        if start_day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday'] and end_day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
            if (start_time.hour == 0 and start_time.minute == 0 and start_time.second == 0) and (end_time.hour == 10 and end_time.minute == 0 and end_time.second == 0):
                row['discount_factor'] = 0.8
            elif (start_time.hour == 10 and start_time.minute == 0 and start_time.second == 0) and (end_time.hour == 18 and end_time.minute == 0 and end_time.second == 0):
                row['discount_factor'] = 1.2
            elif (start_time.hour == 18 and start_time.minute == 0 and start_time.second == 0) and (end_time.hour == 23 and end_time.minute == 59 and end_time.second == 59):
                row['discount_factor'] = 0.8
        elif start_day in ['Saturday', 'Sunday'] and end_day in ['Saturday', 'Sunday']:
            row['discount_factor'] = 0.7

        return row

    df = df.apply(apply_discount_factor, axis=1)
    df.to_csv('updated_carbon_emission_data.csv', index=False)

In [42]:
df = pd.read_csv('dataset-3.csv')
unrolled_df = unroll_distance_matrix(df)
desired_id_starts = ['a', 'b', 'c']
df_with_time_based_toll_rates = unrolled_df[unrolled_df['id_start'].isin(desired_id_starts)]
result_df = calculate_time_based_toll_rates(df_with_time_based_toll_rates)
print(result_df)

KeyError: 'start_time'